# Setup 

### Import libraries

In [1]:
import yaml 
from pathlib import Path
import os
path = Path("C:/Users/HP/Desktop/UNOS data/codes/simulation-waitline-unos").resolve()  # Get the absolute path
os.chdir(path)
import prediction as pred 
from data import *
from simulation import * 
import copy

Load data from UNOS and dictionaries to map ethnicities and blood type compatibility

In [2]:
df_base = load_data()   
yaml_file_path = "C:/Users/HP/Desktop/UNOS data/codes/simulation-waitline-unos/mapping_data.yaml"
with open(yaml_file_path, "r") as file:
    loaded_data = yaml.safe_load(file)
ethcat_mapping = loaded_data["ethcat_mapping"]
blood_type_compatibility = loaded_data["blood_type_compatibility"]

Get proportions by gender from UNOS sample

In [3]:
real_gender_df = df_base.GENDER.value_counts(normalize=True).reset_index()
real_gender_df.columns = ['gender', 'sample_proportion']
real_gender_df

,gender,sample_proportion
0,M,0.64827
1,F,0.35173


Get proportions by ethnicity from UNOS sample

In [4]:
real_ethcat_df = df_base.ETHCAT.value_counts(normalize=True).reset_index()
real_ethcat_df.ETHCAT = real_ethcat_df.ETHCAT.map(ethcat_mapping)
real_ethcat_df.columns = ['ethcat', 'sample_proportion']
real_ethcat_df

,ethcat,sample_proportion
0,"White, Non-Hispanic",0.439416
1,"Black, Non-Hispanic",0.325258
2,Hispanic/Latino,0.159551
3,"Asian, Non-Hispanic",0.053213
4,"Amer Ind/Alaska Native, Non-Hispanic",0.013618
5,"Multiracial, Non-Hispanic",0.004764
6,"Native Hawaiian/Other Pacific Islander, Non-Hi...",0.004180


Load predictors and create events

In [5]:
decision_tree = pred.SurvivalPredictionModel(model_path='models\decision_tree_0912.pkl')
cox_prop = pred.SurvivalPredictionModel(model_path='models\coxprop_0912.pkl')
random_forest = pred.SurvivalPredictionModel(model_path=r'models\forest_0912.pkl')

replicates = 1
events_per_replication =create_list_of_events(n_events = 10000, replications=replicates)

events_per_replication_p1 = copy.deepcopy(events_per_replication)
events_per_replication_p2 = copy.deepcopy(events_per_replication)
events_per_replication_p3 = copy.deepcopy(events_per_replication)



<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\c'
C:\Users\HP\AppData\Local\Temp\ipykernel_25484\1888088931.py:1: SyntaxWarning: invalid escape sequence '\d'
  decision_tree = pred.SurvivalPredictionModel(model_path='models\decision_tree_0912.pkl')
C:\Users\HP\AppData\Local\Temp\ipykernel_25484\1888088931.py:2: SyntaxWarning: invalid escape sequence '\c'
  cox_prop = pred.SurvivalPredictionModel(model_path='models\coxprop_0912.pkl')


models\decision_tree_0912.pkl
ok
Model loaded successfully from models\decision_tree_0912.pkl.
models\coxprop_0912.pkl
ok
Model loaded successfully from models\coxprop_0912.pkl.
models\forest_0912.pkl
ok
Model loaded successfully from models\forest_0912.pkl.


# Survival Decision tree

## Policy 1

In [ ]:
average_waiting_times, number_of_matches_list, results_p1_no, results_p1_ll, results_p1_fl = run_simulation(events_per_replication_p1,  replicates, decision_tree, policy='p1', verbose=True)

Replicate # 0


Progress replication 0:   0%|          | 7.491039175972739/10957.5 [00:00<02:42, 67.51time unit/s]

Recipient ID: 5424
Predictors: [ 1.80118333 69.          1.          0.        ]
ethcat: 1
gender: M


Progress replication 0:   0%|          | 18.6713350239477/10957.5 [00:00<06:07, 29.74time unit/s]  

Recipient ID: 13949
Predictors: [ 2.08226097 49.          0.          0.        ]
ethcat: 2
gender: M
Recipient ID: 1373
Predictors: [ 1.73532552 57.          1.          0.        ]
ethcat: 2
gender: F


...

Recipient ID: 9896
Predictors: [ 0. 58.  0.  1.]
ethcat: 1
gender: M
Recipient ID: 14674
Predictors: [ 0. 63.  1.  1.]
ethcat: 5
gender: M
Recipient ID: 22240
Predictors: [ 1.66274834 66.          1.          0.        ]
ethcat: 4
gender: M


Progress replication 0:   2%|▏         | 231.9884110922534/10957.5 [00:06<04:49, 37.09time unit/s] 

Recipient ID: 16121
Predictors: [ 0. 57.  0.  1.]
ethcat: 1
gender: M
Recipient ID: 15213
Predictors: [ 0.48503384 59.          1.          0.        ]
ethcat: 1
gender: F


Progress replication 0:   2%|▏         | 232.30538109068604/10957.5 [00:06<04:49, 37.02time unit/s]


KeyboardInterrupt: 

### recipients who receive a kidney

In [ ]:
df_waiting_time_no, df_gender_no = get_summary_tables(results_p1_no, ethcat_mapping, real_ethcat_df, real_gender_df)

In [ ]:

df_waiting_time_no.to_csv('results/decision_tree_p1_eth_no.csv') 
df_waiting_time_no

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",1262.767,NaN,19.0,NaN,2957.907,NaN,0.014,0.014
1,"Asian, Non-Hispanic",1415.125,NaN,64.0,NaN,2847.625,NaN,0.046,0.053
2,"Black, Non-Hispanic",1235.944,NaN,433.0,NaN,2858.107,NaN,0.314,0.325
3,Hispanic/Latino,1235.456,NaN,220.0,NaN,2821.881,NaN,0.159,0.160
4,"Multiracial, Non-Hispanic",1069.802,NaN,7.0,NaN,2985.794,NaN,0.005,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1487.738,NaN,6.0,NaN,2852.112,NaN,0.004,0.004
6,"White, Non-Hispanic",1203.199,NaN,632.0,NaN,2847.473,NaN,0.458,0.439


In [ ]:

df_gender_no.to_csv('results/decision_tree_p1_gender_no.csv')
df_gender_no

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,1231.202622,NaN,485.0,NaN,2847.106121,NaN,0.351195,0.35173
1,M,1229.049386,NaN,896.0,NaN,2849.991127,NaN,0.648805,0.64827


### patients who leave the waitlist

In [ ]:
df_waiting_time_ll, df_gender_ll = get_summary_tables(results_p1_ll, ethcat_mapping, real_ethcat_df, real_gender_df)

In [ ]:
df_waiting_time_ll.to_csv('results/decision_tree_p1_eth_ll.csv')
df_waiting_time_ll

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",573.067,NaN,6.0,NaN,0.0,NaN,0.011,0.014
1,"Asian, Non-Hispanic",850.246,NaN,25.0,NaN,0.0,NaN,0.046,0.053
2,"Black, Non-Hispanic",717.779,NaN,191.0,NaN,0.0,NaN,0.349,0.325
3,Hispanic/Latino,599.121,NaN,83.0,NaN,0.0,NaN,0.152,0.160
4,"Multiracial, Non-Hispanic",823.816,NaN,3.0,NaN,0.0,NaN,0.005,0.005
5,"White, Non-Hispanic",642.260,NaN,239.0,NaN,0.0,NaN,0.437,0.439


In [ ]:
df_gender_ll.to_csv('results/decision_tree_p1_gender_ll.csv')
df_gender_ll

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,720.167837,NaN,202.0,NaN,0.0,NaN,0.369287,0.35173
1,M,643.521669,NaN,345.0,NaN,0.0,NaN,0.630713,0.64827


### patients who stay in final list

In [ ]:
df_waiting_time_fl, df_gender_fl = get_summary_tables(results_p1_fl, ethcat_mapping, real_ethcat_df, real_gender_df)

In [ ]:
df_waiting_time_fl.to_csv('results/decision_tree_p1_eth_fl.csv')
df_waiting_time_fl

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",589.030,NaN,3.0,NaN,0.0,NaN,0.011,0.014
1,"Asian, Non-Hispanic",785.377,NaN,20.0,NaN,0.0,NaN,0.075,0.053
2,"Black, Non-Hispanic",611.250,NaN,75.0,NaN,0.0,NaN,0.283,0.325
3,Hispanic/Latino,515.713,NaN,37.0,NaN,0.0,NaN,0.140,0.160
4,"Multiracial, Non-Hispanic",390.624,NaN,2.0,NaN,0.0,NaN,0.008,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",755.475,NaN,2.0,NaN,0.0,NaN,0.008,0.004
6,"White, Non-Hispanic",707.205,NaN,126.0,NaN,0.0,NaN,0.475,0.439


In [ ]:
df_gender_fl.to_csv('results/decision_tree_p1_gender_fl.csv')
df_gender_fl

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,691.249742,NaN,106.0,NaN,0.0,NaN,0.4,0.35173
1,M,632.247194,NaN,159.0,NaN,0.0,NaN,0.6,0.64827


In [ ]:
get_probability_by_ethinicy(df_waiting_time_no, df_waiting_time_ll, df_waiting_time_fl).to_csv('results/decision_tree_p1_eth_prob.csv')

## Policy 2

In [ ]:
average_waiting_times, number_of_matches_list, results_p2_no, results_p2_ll, results_p2_fl = run_simulation(events_per_replication_p2,  replicates, decision_tree, policy='p2', verbose=False)

Replicate # 0


Progress replication 0:   0%|          | 7.792137586390645/10957.5 [00:00<02:28, 73.85time unit/s]

Recipient ID: 17460
Predictors: [ 0.51981996 60.          1.          0.        ]
ethcat: 1
gender: F
Recipient ID: 18568
Predictors: [ 0.78559096 66.          0.          0.        ]
ethcat: 1
gender: M
Recipient ID: 873
Predictors: [ 0. 74.  0.  1.]
ethcat: 2
gender: M


Progress replication 0:   0%|          | 31.845475811169994/10957.5 [00:00<01:52, 96.89time unit/s]

Recipient ID: 19143
Predictors: [ 0. 63.  1.  1.]
ethcat: 5
gender: M
Recipient ID: 17119
Predictors: [ 1.36081859 52.          1.          0.        ]
ethcat: 5
gender: M
Recipient ID: 2058
Predictors: [ 1.68177449 70.          0.          0.        ]
ethcat: 1
gender: F
Recipient ID: 10227
Predictors: [ 1.42998697 53.          1.          0.        ]
ethcat: 1
gender: M
Recipient ID: 12652
Predictors: [ 0.93487132 69.          1.          0.        ]
ethcat: 4
gender: M
Recipient ID: 10475
Predictors: [ 1.44041562 59.          1.          0.        ]
ethcat: 1
gender: M
Recipient ID: 836
Predictors: [ 1.66378608 71.          1.          0.        ]
ethcat: 2
gender: F


...

Recipient ID: 9613
Predictors: [ 2.21142099 65.          1.          0.        ]
ethcat: 4
gender: F
Recipient ID: 13454
Predictors: [ 1.73194019 69.          1.          0.        ]
ethcat: 4
gender: M
Recipient ID: 7258
Predictors: [ 2.21680499 62.          0.          0.        ]
ethcat: 5
gender: M
Recipient ID: 21365
Predictors: [ 0. 42.  0.  1.]
ethcat: 2
gender: M
Recipient ID: 9219
Predictors: [ 1.693925 44.        1.        0.      ]
ethcat: 6
gender: M
Recipient ID: 17444
Predictors: [ 2.59216475 61.          1.          0.        ]
ethcat: 2
gender: M


Progress replication 0: 100%|█████████▉| 10954.227258955241/10957.5 [01:41<00:00, 122.18time unit/s]

Recipient ID: 16915
Predictors: [ 1.27662768 71.          1.          0.        ]
ethcat: 2
gender: M
Recipient ID: 10357
Predictors: [ 2.18374914 66.          1.          0.        ]
ethcat: 4
gender: F
Recipient ID: 6979
Predictors: [ 1.838015 67.        1.        0.      ]
ethcat: 2
gender: F
Recipient ID: 9239
Predictors: [ 1.88441123 57.          0.          0.        ]
ethcat: 4
gender: M


Progress replication 0: 10958.705207421935time unit [01:41, 107.70time unit/s]                      

Recipient ID: 6853
Predictors: [ 1.681265 75.        1.        0.      ]
ethcat: 4
gender: M


### recipients who receive a kidney

In [ ]:
df_waiting_time_no, df_gender_no = get_summary_tables(results_p2_no, ethcat_mapping, real_ethcat_df, real_gender_df)

In [ ]:
df_waiting_time_no.to_csv('results/decision_tree_p2_eth_no.csv') 
df_waiting_time_no

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",989.338,NaN,26.0,NaN,2804.550,NaN,0.019,0.014
1,"Asian, Non-Hispanic",1199.176,NaN,67.0,NaN,2825.870,NaN,0.049,0.053
2,"Black, Non-Hispanic",1155.389,NaN,442.0,NaN,2841.677,NaN,0.320,0.325
3,Hispanic/Latino,1118.086,NaN,227.0,NaN,2862.188,NaN,0.164,0.160
4,"Multiracial, Non-Hispanic",1193.380,NaN,7.0,NaN,2858.557,NaN,0.005,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1059.781,NaN,8.0,NaN,2676.429,NaN,0.006,0.004
6,"White, Non-Hispanic",1153.245,NaN,603.0,NaN,2856.152,NaN,0.437,0.439


In [ ]:
df_gender_no.to_csv('results/decision_tree_p2_gender_no.csv')
df_gender_no

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,1134.490248,NaN,513.0,NaN,2843.749395,NaN,0.371739,0.35173
1,M,1154.325509,NaN,867.0,NaN,2852.164941,NaN,0.628261,0.64827


### patients who leave the waitlist

In [ ]:
df_waiting_time_ll, df_gender_ll = get_summary_tables(results_p2_ll, ethcat_mapping, real_ethcat_df, real_gender_df)

In [ ]:
df_waiting_time_ll.to_csv('results/decision_tree_p2_eth_ll.csv') 
df_waiting_time_ll

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",420.603,NaN,12.0,NaN,0.0,NaN,0.023,0.014
1,"Asian, Non-Hispanic",549.576,NaN,33.0,NaN,0.0,NaN,0.063,0.053
2,"Black, Non-Hispanic",653.723,NaN,157.0,NaN,0.0,NaN,0.298,0.325
3,Hispanic/Latino,597.144,NaN,88.0,NaN,0.0,NaN,0.167,0.160
4,"Multiracial, Non-Hispanic",692.161,NaN,4.0,NaN,0.0,NaN,0.008,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",774.346,NaN,4.0,NaN,0.0,NaN,0.008,0.004
6,"White, Non-Hispanic",569.779,NaN,229.0,NaN,0.0,NaN,0.435,0.439


In [ ]:
df_gender_ll.to_csv('results/decision_tree_p2_gender_ll.csv')
df_gender_ll

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,617.259575,NaN,198.0,NaN,0.0,NaN,0.375712,0.35173
1,M,585.089434,NaN,329.0,NaN,0.0,NaN,0.624288,0.64827


### patients who stay in final list

In [ ]:
df_waiting_time_fl, df_gender_fl = get_summary_tables(results_p2_fl, ethcat_mapping, real_ethcat_df, real_gender_df)

In [ ]:
df_waiting_time_fl.to_csv('results/decision_tree_p2_eth_fl.csv')
df_waiting_time_fl

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",700.353,NaN,3.0,NaN,0.0,NaN,0.012,0.014
1,"Asian, Non-Hispanic",519.186,NaN,15.0,NaN,0.0,NaN,0.059,0.053
2,"Black, Non-Hispanic",629.437,NaN,95.0,NaN,0.0,NaN,0.375,0.325
3,Hispanic/Latino,594.421,NaN,43.0,NaN,0.0,NaN,0.170,0.160
4,"Multiracial, Non-Hispanic",1063.665,NaN,1.0,NaN,0.0,NaN,0.004,0.005
5,"White, Non-Hispanic",665.559,NaN,96.0,NaN,0.0,NaN,0.379,0.439


In [ ]:
df_gender_fl.to_csv('results/decision_tree_p2_gender_fl.csv')
df_gender_fl

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,669.523222,NaN,90.0,NaN,0.0,NaN,0.355731,0.35173
1,M,613.163428,NaN,163.0,NaN,0.0,NaN,0.644269,0.64827


In [ ]:
get_probability_by_ethinicy(df_waiting_time_no, df_waiting_time_ll, df_waiting_time_fl).to_csv('results/decision_tree_p2_eth_prob.csv')

## Policy 3

In [ ]:
average_waiting_times, number_of_matches_list, results_p3_no, results_p3_ll, results_p3_fl = run_simulation(events_per_replication_p3,  replicates, random_forest, policy='p3', verbose=False)

Replicate # 0


Progress replication 0:   0%|          | 0/10957.5 [00:00<?, ?time unit/s]

Recipient ID: 471
Predictors: [ 1.59176034 37.          0.          0.        ]
ethcat: 1
gender: F


Progress replication 0:   0%|          | 12.152406625153805/10957.5 [00:03<1:06:00,  2.76time unit/s]

Recipient ID: 18943
Predictors: [ 1.06095093 69.          1.          0.        ]
ethcat: 1
gender: M
Recipient ID: 9199
Predictors: [ 0. 74.  0.  1.]
ethcat: 1
gender: M


...

Recipient ID: 6877
Predictors: [ 0. 72.  1.  1.]
ethcat: 2
gender: M


Progress replication 0:  71%|███████   | 7773.380414897056/10957.5 [17:16<09:13,  5.76time unit/s]

Recipient ID: 3191
Predictors: [ 0.84377156 63.          1.          0.        ]
ethcat: 1
gender: M
Recipient ID: 15391
Predictors: [ 1.72951507 63.          1.          0.        ]
ethcat: 1
gender: M


Progress replication 0:  71%|███████   | 7789.611118958051/10957.5 [17:19<10:59,  4.80time unit/s]

Recipient ID: 4747
Predictors: [ 1.12695427 73.          1.          0.        ]
ethcat: 1
gender: M
Recipient ID: 11805
Predictors: [ 1.92516252 54.          1.          0.        ]
ethcat: 1
gender: M


### recipients who receive a kidney

In [ ]:
df_waiting_time_no, df_gender_no = get_summary_tables(results_p3_no, ethcat_mapping, real_ethcat_df, real_gender_df)

In [ ]:
df_waiting_time_no.to_csv('results/decision_tree_p3_eth_no.csv')  
df_waiting_time_no

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",1009.004,NaN,16.0,NaN,2998.484,NaN,0.012,0.014
1,"Asian, Non-Hispanic",1107.000,NaN,62.0,NaN,2971.588,NaN,0.045,0.053
2,"Black, Non-Hispanic",1125.292,NaN,458.0,NaN,2954.664,NaN,0.332,0.325
3,Hispanic/Latino,1057.151,NaN,242.0,NaN,2966.092,NaN,0.175,0.160
4,"Multiracial, Non-Hispanic",1107.504,NaN,9.0,NaN,2965.515,NaN,0.007,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1009.828,NaN,6.0,NaN,3047.545,NaN,0.004,0.004
6,"White, Non-Hispanic",1093.481,NaN,588.0,NaN,2974.046,NaN,0.426,0.439


In [ ]:
df_gender_no.to_csv('results/decision_tree_p3_gender_no.csv')
df_gender_no

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,1111.919977,NaN,510.0,NaN,2974.115613,NaN,0.369298,0.35173
1,M,1088.297204,NaN,871.0,NaN,2962.295785,NaN,0.630702,0.64827


### patients who leave the waitlist

In [ ]:
df_waiting_time_ll, df_gender_ll = get_summary_tables(results_p3_ll, ethcat_mapping, real_ethcat_df, real_gender_df)

In [ ]:
df_waiting_time_ll.to_csv('results/decision_tree_p3_eth_ll.csv')
df_waiting_time_ll

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",460.861,NaN,8.0,NaN,0.0,NaN,0.015,0.014
1,"Asian, Non-Hispanic",624.165,NaN,29.0,NaN,0.0,NaN,0.056,0.053
2,"Black, Non-Hispanic",529.784,NaN,168.0,NaN,0.0,NaN,0.324,0.325
3,Hispanic/Latino,555.653,NaN,95.0,NaN,0.0,NaN,0.183,0.160
4,"Multiracial, Non-Hispanic",238.575,NaN,1.0,NaN,0.0,NaN,0.002,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",301.240,NaN,1.0,NaN,0.0,NaN,0.002,0.004
6,"White, Non-Hispanic",571.016,NaN,216.0,NaN,0.0,NaN,0.417,0.439


In [ ]:
df_gender_ll.to_csv('results/decision_tree_p3_gender_ll.csv')
df_gender_ll

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,547.126196,NaN,178.0,NaN,0.0,NaN,0.343629,0.35173
1,M,559.027129,NaN,340.0,NaN,0.0,NaN,0.656371,0.64827


### patients who stay in final list

In [ ]:
df_waiting_time_fl, df_gender_fl = get_summary_tables(results_p3_fl, ethcat_mapping, real_ethcat_df, real_gender_df)

In [ ]:
df_waiting_time_fl.to_csv('results/decision_tree_p3_eth_fl.csv')
df_waiting_time_fl

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",537.791,NaN,3.0,NaN,0.0,NaN,0.012,0.014
1,"Asian, Non-Hispanic",636.947,NaN,7.0,NaN,0.0,NaN,0.029,0.053
2,"Black, Non-Hispanic",521.030,NaN,94.0,NaN,0.0,NaN,0.384,0.325
3,Hispanic/Latino,667.283,NaN,44.0,NaN,0.0,NaN,0.180,0.160
4,"Multiracial, Non-Hispanic",247.441,NaN,2.0,NaN,0.0,NaN,0.008,0.005
5,"White, Non-Hispanic",680.253,NaN,95.0,NaN,0.0,NaN,0.388,0.439


In [ ]:
df_gender_fl.to_csv('results/decision_tree_p3_gender_fl.csv')
df_gender_fl

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,562.906555,NaN,84.0,NaN,0.0,NaN,0.342857,0.35173
1,M,635.056122,NaN,161.0,NaN,0.0,NaN,0.657143,0.64827


In [ ]:
get_probability_by_ethinicy(df_waiting_time_no, df_waiting_time_ll, df_waiting_time_fl).to_csv('results/decision_tree_p3_eth_prob.csv')

# Random survival forest

In [ ]:
events_per_replication_p1 = copy.deepcopy(events_per_replication)
events_per_replication_p2 = copy.deepcopy(events_per_replication)
events_per_replication_p3 = copy.deepcopy(events_per_replication)

## Policy 1

In [ ]:
average_waiting_times, number_of_matches_list, results_p1_no, results_p1_ll, results_p1_fl = run_simulation(events_per_replication_p1,  
                                                                                                            replicates, random_forest, 
                                                                                                            policy='p1', verbose=False)

Replicate # 0


Progress replication 0: 10959.15887291045time unit [09:30, 19.20time unit/s]                       


### recipients who receive a kidney

In [ ]:
df_waiting_time_no, df_gender_no = get_summary_tables(results_p1_no, ethcat_mapping, real_ethcat_df, real_gender_df)

In [ ]:
df_waiting_time_no.to_csv('results/random_forest_p1_eth_no.csv') 
df_waiting_time_no

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",533.733,NaN,13.0,NaN,2895.911,NaN,0.009,0.014
1,"Asian, Non-Hispanic",720.382,NaN,62.0,NaN,2923.898,NaN,0.045,0.053
2,"Black, Non-Hispanic",718.357,NaN,458.0,NaN,2962.810,NaN,0.332,0.325
3,Hispanic/Latino,748.942,NaN,219.0,NaN,2981.242,NaN,0.159,0.160
4,"Multiracial, Non-Hispanic",1102.513,NaN,6.0,NaN,3026.580,NaN,0.004,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1650.771,NaN,6.0,NaN,2915.049,NaN,0.004,0.004
6,"White, Non-Hispanic",593.724,NaN,617.0,NaN,2975.808,NaN,0.447,0.439


In [ ]:
df_gender_no.to_csv('results/random_forest_p1_gender_no.csv')
df_gender_no

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,635.060411,NaN,513.0,NaN,2965.911841,NaN,0.37147,0.35173
1,M,693.190903,NaN,868.0,NaN,2971.195835,NaN,0.62853,0.64827


### patients who leave the waitlist

In [ ]:
df_waiting_time_ll, df_gender_ll = get_summary_tables(results_p1_ll, ethcat_mapping, real_ethcat_df, real_gender_df)

In [ ]:
df_waiting_time_ll.to_csv('results/random_forest_p1_eth_ll.csv')
df_waiting_time_ll

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",1185.348,NaN,11.0,NaN,0.0,NaN,0.021,0.014
1,"Asian, Non-Hispanic",1352.222,NaN,27.0,NaN,0.0,NaN,0.051,0.053
2,"Black, Non-Hispanic",1601.877,NaN,181.0,NaN,0.0,NaN,0.343,0.325
3,Hispanic/Latino,1371.096,NaN,120.0,NaN,0.0,NaN,0.228,0.160
4,"Multiracial, Non-Hispanic",1819.000,NaN,2.0,NaN,0.0,NaN,0.004,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",700.181,NaN,2.0,NaN,0.0,NaN,0.004,0.004
6,"White, Non-Hispanic",1622.916,NaN,184.0,NaN,0.0,NaN,0.349,0.439


In [ ]:
df_gender_ll.to_csv('results/random_forest_p1_gender_ll.csv')
df_gender_ll

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,1605.062166,NaN,183.0,NaN,0.0,NaN,0.347249,0.35173
1,M,1494.036510,NaN,344.0,NaN,0.0,NaN,0.652751,0.64827


### patients who stay in final list

In [ ]:
df_waiting_time_fl, df_gender_fl = get_summary_tables(results_p1_fl, ethcat_mapping, real_ethcat_df, real_gender_df)

In [ ]:
df_waiting_time_fl.to_csv('results/random_forest_p1_eth_fl.csv')
df_waiting_time_fl

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",984.588,NaN,4.0,NaN,0.0,NaN,0.017,0.014
1,"Asian, Non-Hispanic",2420.620,NaN,11.0,NaN,0.0,NaN,0.046,0.053
2,"Black, Non-Hispanic",1565.041,NaN,88.0,NaN,0.0,NaN,0.368,0.325
3,Hispanic/Latino,1799.815,NaN,49.0,NaN,0.0,NaN,0.205,0.160
4,"Multiracial, Non-Hispanic",2087.359,NaN,4.0,NaN,0.0,NaN,0.017,0.005
5,"White, Non-Hispanic",1793.391,NaN,83.0,NaN,0.0,NaN,0.347,0.439


In [ ]:
df_gender_fl.to_csv('results/random_forest_p1_gender_fl.csv')
df_gender_fl

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,1896.212771,NaN,74.0,NaN,0.0,NaN,0.309623,0.35173
1,M,1656.732619,NaN,165.0,NaN,0.0,NaN,0.690377,0.64827


In [ ]:
get_probability_by_ethinicy(df_waiting_time_no, df_waiting_time_ll, df_waiting_time_fl).to_csv('results/random_forest_p1_eth_prob.csv')

## Policy 2

In [ ]:
average_waiting_times, number_of_matches_list, results_p2_no, results_p2_ll, results_p2_fl = run_simulation(events_per_replication_p2,  replicates, random_forest, policy='p2', verbose=False)

Replicate # 0


Progress replication 0: 10959.15887291045time unit [01:15, 144.40time unit/s]                       


### recipients who receive a kidney

In [ ]:
df_waiting_time_no, df_gender_no = get_summary_tables(results_p2_no, ethcat_mapping, real_ethcat_df, real_gender_df)

In [ ]:
df_waiting_time_no.to_csv('results/random_forest_p2_eth_no.csv')
df_waiting_time_no

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",1008.641,NaN,16.0,NaN,3022.626,NaN,0.012,0.014
1,"Asian, Non-Hispanic",1080.577,NaN,62.0,NaN,2963.084,NaN,0.045,0.053
2,"Black, Non-Hispanic",1120.023,NaN,456.0,NaN,2942.224,NaN,0.330,0.325
3,Hispanic/Latino,1059.806,NaN,244.0,NaN,2958.695,NaN,0.177,0.160
4,"Multiracial, Non-Hispanic",1089.325,NaN,9.0,NaN,3017.590,NaN,0.007,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1024.543,NaN,6.0,NaN,3006.033,NaN,0.004,0.004
6,"White, Non-Hispanic",1106.316,NaN,588.0,NaN,2972.769,NaN,0.426,0.439


In [ ]:
df_gender_no.to_csv('results/random_forest_p2_gender_no.csv')
df_gender_no

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,1118.686414,NaN,506.0,NaN,2962.463991,NaN,0.366401,0.35173
1,M,1088.990643,NaN,875.0,NaN,2959.800003,NaN,0.633599,0.64827


### patients who leave the waitlist

In [ ]:
df_waiting_time_ll, df_gender_ll = get_summary_tables(results_p2_ll, ethcat_mapping, real_ethcat_df, real_gender_df)

In [ ]:
df_waiting_time_ll.to_csv('results/random_forest_p2_eth_ll.csv')
df_waiting_time_ll

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",460.861,NaN,8.0,NaN,0.0,NaN,0.015,0.014
1,"Asian, Non-Hispanic",634.286,NaN,30.0,NaN,0.0,NaN,0.058,0.053
2,"Black, Non-Hispanic",523.739,NaN,166.0,NaN,0.0,NaN,0.321,0.325
3,Hispanic/Latino,525.302,NaN,93.0,NaN,0.0,NaN,0.180,0.160
4,"Multiracial, Non-Hispanic",238.575,NaN,1.0,NaN,0.0,NaN,0.002,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",301.240,NaN,1.0,NaN,0.0,NaN,0.002,0.004
6,"White, Non-Hispanic",575.954,NaN,218.0,NaN,0.0,NaN,0.422,0.439


In [ ]:
df_gender_ll.to_csv('results/random_forest_p2_gender_ll.csv')
df_gender_ll

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,538.881565,NaN,179.0,NaN,0.0,NaN,0.346228,0.35173
1,M,556.648640,NaN,338.0,NaN,0.0,NaN,0.653772,0.64827


### patients who stay in final list

In [ ]:
df_waiting_time_fl, df_gender_fl = get_summary_tables(results_p2_fl, ethcat_mapping, real_ethcat_df, real_gender_df)

In [ ]:
df_waiting_time_fl.to_csv('results/random_forest_p2_eth_fl.csv')
df_waiting_time_fl

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",537.791,NaN,3.0,NaN,0.0,NaN,0.012,0.014
1,"Asian, Non-Hispanic",636.947,NaN,7.0,NaN,0.0,NaN,0.028,0.053
2,"Black, Non-Hispanic",547.189,NaN,95.0,NaN,0.0,NaN,0.386,0.325
3,Hispanic/Latino,675.587,NaN,45.0,NaN,0.0,NaN,0.183,0.160
4,"Multiracial, Non-Hispanic",247.441,NaN,2.0,NaN,0.0,NaN,0.008,0.005
5,"White, Non-Hispanic",670.977,NaN,94.0,NaN,0.0,NaN,0.382,0.439


In [ ]:
df_gender_fl.to_csv('results/random_forest_p2_gender_fl.csv')
df_gender_fl

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,562.906555,NaN,84.0,NaN,0.0,NaN,0.341463,0.35173
1,M,646.536487,NaN,162.0,NaN,0.0,NaN,0.658537,0.64827


In [ ]:
get_probability_by_ethinicy(df_waiting_time_no, df_waiting_time_ll, df_waiting_time_fl).to_csv('results/random_forest_p2_eth_prob.csv')

## Policy 3

### recipients who receive a kidney

In [ ]:
average_waiting_times, number_of_matches_list, results_p3_no, results_p3_ll, results_p3_fl = run_simulation(events_per_replication_p3,  replicates, random_forest, policy='p3', verbose=False)

Replicate # 0


Progress replication 0: 10959.15887291045time unit [18:13, 10.02time unit/s]                         


In [ ]:
df_waiting_time_no, df_gender_no = get_summary_tables(results_p3_no, ethcat_mapping, real_ethcat_df, real_gender_df)

In [ ]:
df_waiting_time_no.to_csv('results/random_forest_p3_eth_no.csv')
df_waiting_time_no

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",1009.004,NaN,16.0,NaN,2998.484,NaN,0.012,0.014
1,"Asian, Non-Hispanic",1107.000,NaN,62.0,NaN,2971.588,NaN,0.045,0.053
2,"Black, Non-Hispanic",1125.292,NaN,458.0,NaN,2954.664,NaN,0.332,0.325
3,Hispanic/Latino,1057.151,NaN,242.0,NaN,2966.092,NaN,0.175,0.160
4,"Multiracial, Non-Hispanic",1107.504,NaN,9.0,NaN,2965.515,NaN,0.007,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1009.828,NaN,6.0,NaN,3047.545,NaN,0.004,0.004
6,"White, Non-Hispanic",1093.481,NaN,588.0,NaN,2974.046,NaN,0.426,0.439


In [ ]:
df_gender_no.to_csv('results/random_forest_p3_gender_no.csv')
df_gender_no

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,1111.919977,NaN,510.0,NaN,2974.115613,NaN,0.369298,0.35173
1,M,1088.297204,NaN,871.0,NaN,2962.295785,NaN,0.630702,0.64827


### patients who leave the waitlist

In [ ]:
df_waiting_time_ll, df_gender_ll = get_summary_tables(results_p3_ll, ethcat_mapping, real_ethcat_df, real_gender_df)

In [ ]:
df_waiting_time_ll.to_csv('results/random_forest_p3_eth_ll.csv')
df_waiting_time_ll

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",460.861,NaN,8.0,NaN,0.0,NaN,0.015,0.014
1,"Asian, Non-Hispanic",624.165,NaN,29.0,NaN,0.0,NaN,0.056,0.053
2,"Black, Non-Hispanic",529.784,NaN,168.0,NaN,0.0,NaN,0.324,0.325
3,Hispanic/Latino,555.653,NaN,95.0,NaN,0.0,NaN,0.183,0.160
4,"Multiracial, Non-Hispanic",238.575,NaN,1.0,NaN,0.0,NaN,0.002,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",301.240,NaN,1.0,NaN,0.0,NaN,0.002,0.004
6,"White, Non-Hispanic",571.016,NaN,216.0,NaN,0.0,NaN,0.417,0.439


In [ ]:
df_gender_ll.to_csv('results/random_forest_p3_gender_ll.csv')
df_gender_ll

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,547.126196,NaN,178.0,NaN,0.0,NaN,0.343629,0.35173
1,M,559.027129,NaN,340.0,NaN,0.0,NaN,0.656371,0.64827


### patients who stay in final list

In [ ]:
df_waiting_time_fl, df_gender_fl = get_summary_tables(results_p3_fl, ethcat_mapping, real_ethcat_df, real_gender_df)

In [ ]:
df_waiting_time_fl.to_csv('results/random_forest_p3_eth_fl.csv')
df_waiting_time_fl

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",537.791,NaN,3.0,NaN,0.0,NaN,0.012,0.014
1,"Asian, Non-Hispanic",636.947,NaN,7.0,NaN,0.0,NaN,0.029,0.053
2,"Black, Non-Hispanic",521.030,NaN,94.0,NaN,0.0,NaN,0.384,0.325
3,Hispanic/Latino,667.283,NaN,44.0,NaN,0.0,NaN,0.180,0.160
4,"Multiracial, Non-Hispanic",247.441,NaN,2.0,NaN,0.0,NaN,0.008,0.005
5,"White, Non-Hispanic",680.253,NaN,95.0,NaN,0.0,NaN,0.388,0.439


In [ ]:
df_gender_fl.to_csv('results/random_forest_p3_gender_fl.csv')
df_gender_fl

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,562.906555,NaN,84.0,NaN,0.0,NaN,0.342857,0.35173
1,M,635.056122,NaN,161.0,NaN,0.0,NaN,0.657143,0.64827


In [ ]:
get_probability_by_ethinicy(df_waiting_time_no, df_waiting_time_ll, df_waiting_time_fl).to_csv('results/cox_prop_p1_eth_prob.csv')

# Cox proportional hazard models

In [ ]:
events_per_replication_p1 = copy.deepcopy(events_per_replication)
events_per_replication_p2 = copy.deepcopy(events_per_replication)
events_per_replication_p3 = copy.deepcopy(events_per_replication)

## Policy 1

In [ ]:
average_waiting_times, number_of_matches_list, results_p1_no, results_p1_ll, results_p1_fl = run_simulation(events_per_replication_p1,  
                                                                                                            replicates, cox_prop, 
                                                                                                            policy='p1', verbose=False)

Replicate # 0


Progress replication 0: 10959.15887291045time unit [03:11, 57.13time unit/s]                        


### recipients who receive a kidney

In [ ]:
df_waiting_time_no, df_gender_no = get_summary_tables(results_p1_no, ethcat_mapping, real_ethcat_df, real_gender_df)

In [ ]:
df_waiting_time_no.to_csv('results/cox_prop_p1_eth_no.csv')
df_waiting_time_no

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",33.756,NaN,22.0,NaN,2581.999,NaN,0.016,0.014
1,"Asian, Non-Hispanic",192.454,NaN,60.0,NaN,2633.695,NaN,0.043,0.053
2,"Black, Non-Hispanic",182.022,NaN,495.0,NaN,2667.851,NaN,0.358,0.325
3,Hispanic/Latino,230.845,NaN,246.0,NaN,2615.168,NaN,0.178,0.160
4,"Multiracial, Non-Hispanic",632.089,NaN,9.0,NaN,2350.541,NaN,0.007,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",27.871,NaN,4.0,NaN,2488.246,NaN,0.003,0.004
6,"White, Non-Hispanic",295.788,NaN,545.0,NaN,2644.797,NaN,0.395,0.439


In [ ]:
df_gender_no.to_csv('results/cox_prop_p1_gender_no.csv')
df_gender_no

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,187.407694,NaN,503.0,NaN,2667.309675,NaN,0.364229,0.35173
1,M,264.142756,NaN,878.0,NaN,2630.533727,NaN,0.635771,0.64827


### patients who leave the waitlist

In [ ]:
df_waiting_time_ll, df_gender_ll = get_summary_tables(results_p1_ll, ethcat_mapping, real_ethcat_df, real_gender_df)

In [ ]:
df_waiting_time_ll.to_csv('results/cox_prop_p1_eth_ll.csv')
df_waiting_time_ll

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",1836.527,NaN,1.0,NaN,0.0,NaN,0.002,0.014
1,"Asian, Non-Hispanic",1839.805,NaN,28.0,NaN,0.0,NaN,0.054,0.053
2,"Black, Non-Hispanic",2539.390,NaN,136.0,NaN,0.0,NaN,0.263,0.325
3,Hispanic/Latino,1998.490,NaN,101.0,NaN,0.0,NaN,0.195,0.160
4,"Multiracial, Non-Hispanic",238.575,NaN,1.0,NaN,0.0,NaN,0.002,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",4635.292,NaN,4.0,NaN,0.0,NaN,0.008,0.004
6,"White, Non-Hispanic",2461.863,NaN,246.0,NaN,0.0,NaN,0.476,0.439


In [ ]:
df_gender_ll.to_csv('results/cox_prop_p1_gender_ll.csv')
df_gender_ll

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,2514.451346,NaN,175.0,NaN,0.0,NaN,0.338491,0.35173
1,M,2295.101228,NaN,342.0,NaN,0.0,NaN,0.661509,0.64827


### patients who stay in final list

In [ ]:
df_waiting_time_fl, df_gender_fl = get_summary_tables(results_p1_fl, ethcat_mapping, real_ethcat_df, real_gender_df)

In [ ]:
df_waiting_time_fl.to_csv('results/cox_prop_p1_eth_fl.csv')
df_waiting_time_fl

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",1096.555,NaN,2.0,NaN,0.0,NaN,0.008,0.014
1,"Asian, Non-Hispanic",3423.144,NaN,11.0,NaN,0.0,NaN,0.045,0.053
2,"Black, Non-Hispanic",2289.538,NaN,75.0,NaN,0.0,NaN,0.305,0.325
3,Hispanic/Latino,2647.926,NaN,43.0,NaN,0.0,NaN,0.175,0.160
4,"Multiracial, Non-Hispanic",1082.620,NaN,2.0,NaN,0.0,NaN,0.008,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",7039.137,NaN,1.0,NaN,0.0,NaN,0.004,0.004
6,"White, Non-Hispanic",3210.884,NaN,112.0,NaN,0.0,NaN,0.455,0.439


In [ ]:
df_gender_fl.to_csv('results/cox_prop_p1_gender_fl.csv')
df_gender_fl

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,3429.608062,NaN,87.0,NaN,0.0,NaN,0.353659,0.35173
1,M,2489.757338,NaN,159.0,NaN,0.0,NaN,0.646341,0.64827


In [ ]:
get_probability_by_ethinicy(df_waiting_time_no, df_waiting_time_ll, df_waiting_time_fl).to_csv('results/cox_prop_p1_eth_prob.csv')

## Policy 2

In [ ]:
average_waiting_times, number_of_matches_list, results_p2_no, results_p2_ll, results_p2_fl = run_simulation(events_per_replication_p2,  
                                                                                                            replicates, cox_prop, 
                                                                                                            policy='p2', verbose=False)

Replicate # 0


Progress replication 0: 10959.15887291045time unit [01:09, 158.11time unit/s]                       


### recipients who receive a kidney

In [ ]:
df_waiting_time_no, df_gender_no = get_summary_tables(results_p2_no, ethcat_mapping, real_ethcat_df, real_gender_df)

In [ ]:
df_waiting_time_no.to_csv('results/cox_prop_p2_eth_no.csv')
df_waiting_time_no

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",1008.641,NaN,16.0,NaN,2673.808,NaN,0.012,0.014
1,"Asian, Non-Hispanic",1080.577,NaN,62.0,NaN,2525.588,NaN,0.045,0.053
2,"Black, Non-Hispanic",1120.023,NaN,456.0,NaN,2571.281,NaN,0.330,0.325
3,Hispanic/Latino,1059.806,NaN,244.0,NaN,2541.112,NaN,0.177,0.160
4,"Multiracial, Non-Hispanic",1089.325,NaN,9.0,NaN,2663.454,NaN,0.007,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1024.543,NaN,6.0,NaN,2578.047,NaN,0.004,0.004
6,"White, Non-Hispanic",1106.316,NaN,588.0,NaN,2565.238,NaN,0.426,0.439


In [ ]:
df_gender_no.to_csv('results/cox_prop_p2_gender_no.csv')
df_gender_no

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,1118.686414,NaN,506.0,NaN,2582.319379,NaN,0.366401,0.35173
1,M,1088.990643,NaN,875.0,NaN,2552.055929,NaN,0.633599,0.64827


### patients who leave the waitlist

In [ ]:
df_waiting_time_ll, df_gender_ll = get_summary_tables(results_p2_ll, ethcat_mapping, real_ethcat_df, real_gender_df)

In [ ]:
df_waiting_time_ll.to_csv('results/cox_prop_p2_eth_ll.csv')
df_waiting_time_ll

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",460.861,NaN,8.0,NaN,0.0,NaN,0.015,0.014
1,"Asian, Non-Hispanic",634.286,NaN,30.0,NaN,0.0,NaN,0.058,0.053
2,"Black, Non-Hispanic",523.739,NaN,166.0,NaN,0.0,NaN,0.321,0.325
3,Hispanic/Latino,525.302,NaN,93.0,NaN,0.0,NaN,0.180,0.160
4,"Multiracial, Non-Hispanic",238.575,NaN,1.0,NaN,0.0,NaN,0.002,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",301.240,NaN,1.0,NaN,0.0,NaN,0.002,0.004
6,"White, Non-Hispanic",575.954,NaN,218.0,NaN,0.0,NaN,0.422,0.439


In [ ]:
df_gender_ll.to_csv('results/cox_prop_p2_gender_ll.csv')
df_gender_ll

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,538.881565,NaN,179.0,NaN,0.0,NaN,0.346228,0.35173
1,M,556.648640,NaN,338.0,NaN,0.0,NaN,0.653772,0.64827


### patients who stay in final list

In [ ]:
df_waiting_time_fl, df_gender_fl = get_summary_tables(results_p2_fl, ethcat_mapping, real_ethcat_df, real_gender_df)

In [ ]:
df_waiting_time_fl.to_csv('results/cox_prop_p2_eth_fl.csv')
df_waiting_time_fl

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",537.791,NaN,3.0,NaN,0.0,NaN,0.012,0.014
1,"Asian, Non-Hispanic",636.947,NaN,7.0,NaN,0.0,NaN,0.028,0.053
2,"Black, Non-Hispanic",547.189,NaN,95.0,NaN,0.0,NaN,0.386,0.325
3,Hispanic/Latino,675.587,NaN,45.0,NaN,0.0,NaN,0.183,0.160
4,"Multiracial, Non-Hispanic",247.441,NaN,2.0,NaN,0.0,NaN,0.008,0.005
5,"White, Non-Hispanic",670.977,NaN,94.0,NaN,0.0,NaN,0.382,0.439


In [ ]:
df_gender_fl.to_csv('results/cox_prop_p2_gender_fl.csv')
df_gender_fl

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,562.906555,NaN,84.0,NaN,0.0,NaN,0.341463,0.35173
1,M,646.536487,NaN,162.0,NaN,0.0,NaN,0.658537,0.64827


In [ ]:
get_probability_by_ethinicy(df_waiting_time_no, df_waiting_time_ll, df_waiting_time_fl).to_csv('results/cox_prop_p2_eth_prob.csv')

## Policy 3

In [ ]:
average_waiting_times, number_of_matches_list, results_p3_no, results_p3_ll, results_p3_fl = run_simulation(events_per_replication_p3,  
                                                                                                            replicates, cox_prop, 
                                                                                                            policy='p3', verbose=False)

Replicate # 0


Progress replication 0: 10959.15887291045time unit [10:48, 16.89time unit/s]                       


### recipients who receive a kidney

In [ ]:
df_waiting_time_no, df_gender_no = get_summary_tables(results_p3_no, ethcat_mapping, real_ethcat_df, real_gender_df)

In [ ]:
df_waiting_time_no.to_csv('results/cox_prop_p3_eth_no.csv')
df_waiting_time_no

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",968.348,NaN,14.0,NaN,2606.887,NaN,0.010,0.014
1,"Asian, Non-Hispanic",1154.764,NaN,60.0,NaN,2533.414,NaN,0.043,0.053
2,"Black, Non-Hispanic",1053.270,NaN,463.0,NaN,2611.104,NaN,0.335,0.325
3,Hispanic/Latino,1047.392,NaN,247.0,NaN,2577.282,NaN,0.179,0.160
4,"Multiracial, Non-Hispanic",1089.717,NaN,9.0,NaN,2591.635,NaN,0.007,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1156.680,NaN,4.0,NaN,2694.976,NaN,0.003,0.004
6,"White, Non-Hispanic",1087.490,NaN,584.0,NaN,2581.468,NaN,0.423,0.439


In [ ]:
df_gender_no.to_csv('results/cox_prop_p3_gender_no.csv')
df_gender_no

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,1068.839838,NaN,517.0,NaN,2596.599928,NaN,0.374366,0.35173
1,M,1071.933651,NaN,864.0,NaN,2584.804509,NaN,0.625634,0.64827


### patients who leave the waitlist

In [ ]:
df_waiting_time_ll, df_gender_ll = get_summary_tables(results_p3_ll, ethcat_mapping, real_ethcat_df, real_gender_df)

In [ ]:
df_waiting_time_ll.to_csv('results/cox_prop_p3_eth_ll.csv')
df_waiting_time_ll

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",563.465,NaN,10.0,NaN,0.0,NaN,0.019,0.014
1,"Asian, Non-Hispanic",671.490,NaN,31.0,NaN,0.0,NaN,0.060,0.053
2,"Black, Non-Hispanic",522.478,NaN,159.0,NaN,0.0,NaN,0.307,0.325
3,Hispanic/Latino,561.378,NaN,94.0,NaN,0.0,NaN,0.181,0.160
4,"Multiracial, Non-Hispanic",238.575,NaN,1.0,NaN,0.0,NaN,0.002,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",700.181,NaN,2.0,NaN,0.0,NaN,0.004,0.004
6,"White, Non-Hispanic",576.787,NaN,221.0,NaN,0.0,NaN,0.427,0.439


In [ ]:
df_gender_ll.to_csv('results/cox_prop_p3_gender_ll.csv')
df_gender_ll

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,557.038047,NaN,180.0,NaN,0.0,NaN,0.34749,0.35173
1,M,565.491986,NaN,338.0,NaN,0.0,NaN,0.65251,0.64827


### patients who stay in final list

In [ ]:
df_waiting_time_fl, df_gender_fl = get_summary_tables(results_p3_fl, ethcat_mapping, real_ethcat_df, real_gender_df)

In [ ]:
df_waiting_time_fl.to_csv('results/cox_prop_p3_eth_fl.csv')
df_waiting_time_fl

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",537.791,NaN,3.0,NaN,0.0,NaN,0.012,0.014
1,"Asian, Non-Hispanic",497.270,NaN,8.0,NaN,0.0,NaN,0.033,0.053
2,"Black, Non-Hispanic",564.768,NaN,96.0,NaN,0.0,NaN,0.393,0.325
3,Hispanic/Latino,679.828,NaN,43.0,NaN,0.0,NaN,0.176,0.160
4,"Multiracial, Non-Hispanic",247.441,NaN,2.0,NaN,0.0,NaN,0.008,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1064.119,NaN,1.0,NaN,0.0,NaN,0.004,0.004
6,"White, Non-Hispanic",691.808,NaN,91.0,NaN,0.0,NaN,0.373,0.439


In [ ]:
df_gender_fl.to_csv('results/cox_prop_p3_gender_fl.csv')
df_gender_fl

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,536.102943,NaN,75.0,NaN,0.0,NaN,0.307377,0.35173
1,M,670.696152,NaN,169.0,NaN,0.0,NaN,0.692623,0.64827


In [ ]:
get_probability_by_ethinicy(df_waiting_time_no, df_waiting_time_ll, df_waiting_time_fl).to_csv('results/cox_prop_p3_eth_prob.csv')